In [1]:
import os
import numpy
from typing import Any
from numpy.typing import NDArray
from numpy import (
    int64 as i64,
    uint8 as u8,
    float64 as f64,
)
import cv2
import ipywidgets # type: ignore
from fingerprint import FE_CONFIG, FM_CONFIG, Fingerprint
from features_extraction_utils import (
    RED,
    GREEN,
    BLUE,
    float_slider,
    float_text,
    int_range_slider,
    int_slider,
    show,
    draw_directional_map_lines,
    draw_minutiae,
    draw_minutiae_with_angle,
    draw_mcc_cylinders,
    draw_match_pairs
)
from config import DATASET_DIR_PATH, DATABASE_DIR_PATH, FINGERPRINTS_IMAGE_FILE_EXTENSION


In [2]:
from features_extraction_utils import draw_singularities
from fingerprint import MccReferenceCellCoordinates, Minutiae, Range, Singularities

fingerprint: Fingerprint

db_tag_range = Range[int].new(min = 1, max = 4, step = 1, value = 1)
finger_tag_range = Range[int].new(min = 1, max = 10, step = 1, value = 1)
acquisition_tag_range = Range[int].new(min = 1, max = 3, step = 1, value = 1)

@ipywidgets.interact(
    db_tag = int_slider(range = db_tag_range, description = "db tag"),
    finger_tag = int_slider(range = finger_tag_range, description = "finger tag"),
    acquisition_tag = int_slider(range = acquisition_tag_range, description = "acquisition tag"),
    gradient_sobel_filter_length = int_slider(
        range = FE_CONFIG.gradient_sobel_filter_length,
        description = "gradient sobel filter length",
    ),
    gradient_module_block_length = int_slider(
        range = FE_CONFIG.gradient_module_block_length,
        description = "gradient module block length",
    ),
    segmentation_mask_threshold_scale = float_slider(
        range = FE_CONFIG.segmentation_mask_threshold_scale,
        description = "segmentation mask threshold scale",
    ),
    directional_map_block_length = int_slider(
        range = FE_CONFIG.directional_map_block_length,
        description = "directional map block lenght",
    ),
    directional_map_blur_filter_length = int_slider(
        range = FE_CONFIG.directional_map_blur_filter_length,
        description = "directional map blur length",
    ),
    local_ridge_block_rows = int_slider(
        range = FE_CONFIG.local_ridge_block_rows,
        description = "local ridge block rows",
    ),
    local_ridge_block_columns = int_slider(
        range = FE_CONFIG.local_ridge_block_columns,
        description = "local ridge block columns",
    ),
    gabor_filters_count = int_slider(
        range = FE_CONFIG.gabor_filters_count,
        description = "gabor filters count",
    ),
    # gabor_filters_sigma = float_text(
    #     value = FE_CONFIG.gabor_filters_sigma,
    #     description = "gabor filters sigma",
    # ),
    # gabor_filters_gamma = float_text(
    #     value = FE_CONFIG.gabor_filters_gamma,
    #     description = "gabor filters gamma",
    # ),
    binarization_block_size = int_slider(
        range = FE_CONFIG.binarization_block_size,
        description = "binarization block size",
    ),
    singularities_min_distance_from_border = int_slider(
        range = FE_CONFIG.singularities_min_distance_from_border,
        description = "singularities min distance from border",
    ),
    minutiae_min_distance_from_border = int_slider(
        range = FE_CONFIG.minutiae_min_distance_from_border,
        description = "minutiae min distance from border",
    ),
    minutiae_followed_length = int_range_slider(
        bounds = FE_CONFIG.minutiae_followed_length,
        description = "minutiae followed length",
    ),
    # mcc_gaussian_std = float_text(
    #     value = FE_CONFIG.mcc_gaussian_std,
    #     description = "mcc gaussian std",
    # ),
    # mcc_sigmoid_tau = float_text(
    #     value = FE_CONFIG.mcc_sigmoid_tau,
    #     description = "mcc sigmoid tau",
    # ),
    # mcc_sigmoid_mu = float_text(
    #     value = FE_CONFIG.mcc_sigmoid_mu,
    #     description = "mcc sigmoid mu",
    # ),
) # type: ignore
def extract_features(
    db_tag: int,
    finger_tag: int,
    acquisition_tag: int,
    gradient_sobel_filter_length: int,
    gradient_module_block_length: int,
    segmentation_mask_threshold_scale: float,
    directional_map_block_length: int,
    directional_map_blur_filter_length: int,
    local_ridge_block_rows: int,
    local_ridge_block_columns: int,
    # gabor_filters_count: int,
    # gabor_filters_sigma: float,
    # gabor_filters_gamma: float,
    binarization_block_size: int,
    singularities_min_distance_from_border: int,
    minutiae_min_distance_from_border: int,
    minutiae_followed_length: tuple[int, int],
    # mcc_gaussian_std: float,
    # mcc_sigmoid_tau: float,
    # mcc_sigmoid_mu: float,
) -> None:
    min_minutiae_followed_length, max_minutiae_followed_length = minutiae_followed_length

    fingerprint_file_path = f"{DATASET_DIR_PATH}\\FVC2006\\db{db_tag}_b\\1{finger_tag:02}_{acquisition_tag}{FINGERPRINTS_IMAGE_FILE_EXTENSION}"

    global fingerprint
    fingerprint = Fingerprint(
        file_path = fingerprint_file_path,
        acquisition_tag = str(acquisition_tag),
        gradient_sobel_filter_length = gradient_sobel_filter_length,
        gradient_module_block_length = gradient_module_block_length,
        segmentation_mask_threshold_scale = segmentation_mask_threshold_scale,
        directional_map_block_length = directional_map_block_length,
        directional_map_blur_filter_length = directional_map_blur_filter_length,
        local_ridge_block_rows = local_ridge_block_rows,
        local_ridge_block_columns = local_ridge_block_columns,
        gabor_filters_count = FE_CONFIG.gabor_filters_count.value,
        gabor_filters_sigma = FE_CONFIG.gabor_filters_sigma,
        gabor_filters_gamma = FE_CONFIG.gabor_filters_gamma,
        binarization_block_size = binarization_block_size,
        singularities_min_distance_from_border = singularities_min_distance_from_border,
        minutiae_min_distance_from_border = minutiae_min_distance_from_border,
        minutiae_followed_length_min = min_minutiae_followed_length,
        minutiae_followed_length_max = max_minutiae_followed_length,
        mcc_reference_cell_coordinates = MccReferenceCellCoordinates(
            total_radius = FE_CONFIG.mcc_total_radius,
            circles_radius = FE_CONFIG.mcc_circles_radius,
        ), # TODO(stefano): add sliders for cylinders controls
        mcc_gaussian_std = FE_CONFIG.mcc_gaussian_std,
        mcc_sigmoid_tau = FE_CONFIG.mcc_sigmoid_tau,
        mcc_sigmoid_mu = FE_CONFIG.mcc_sigmoid_mu,
    )

    # show(
    #     (f"Raw fingerprint {fingerprint.raw_fingerprint.shape}", fingerprint.raw_fingerprint),
    #     (f"Normalized {fingerprint.normalized_fingerprint.shape}", fingerprint.normalized_fingerprint),
    #     (f"Normalized negative {fingerprint.normalized_negative_fingerprint.shape}", fingerprint.normalized_negative_fingerprint),
    # )

    # show(
    #     ("Gradient x", fingerprint.gradient_x),
    #     ("Gradient y", fingerprint.gradient_y),
    #     ("Gradient x**2", fingerprint.gradient_x2),
    #     ("Gradient y**2", fingerprint.gradient_y2),

    #     ("Gradient module", fingerprint.gradient_module),

    #     # ("Gradient x**2 filtered", fingerprint.gradient_x2_filtered),
    #     # ("Gradient y**2 filtered", fingerprint.gradient_y2_filtered),
    #     # ("Gradient x*y filtered", fingerprint.gradient_xy_filtered),

    #     # ("Gradient x**2 - y**2 filtered", fingerprint.gradient_x2_minus_y2_filtered),
    #     # ("Gradient 2x*y filtered", fingerprint.gradient_2xy_filtered),

    #     max_images_per_row = 5,
    # )

    # fingerprint_with_directional_lines = draw_directional_map_lines(
    #     fingerprint.normalized_fingerprint,
    #     fingerprint.directional_map,
    #     fingerprint.segmentation_mask,
    #     directional_map_block_length,
    #     RED,
    # )
    # show(
    #     ("Segmentation mask", fingerprint.segmentation_mask),
    #     ("Segmentation mask distance map", fingerprint.segmentation_mask_distance_map),
    #     ("Segmentation mask", cv2.merge((
    #         fingerprint.normalized_fingerprint,
    #         fingerprint.normalized_fingerprint,
    #         fingerprint.segmentation_mask
    #     ))),

    #     ("Directional Map", fingerprint_with_directional_lines),
    # )

    # fingerprint_with_highlighted_ridge_block = cv2.rectangle(
    #     cv2.cvtColor(fingerprint.normalized_fingerprint, cv2.COLOR_GRAY2BGR),
    #     pt1 = (fingerprint.ridge_block_row_start, fingerprint.ridge_block_column_start),
    #     pt2 = (fingerprint.ridge_block_row_end, fingerprint.ridge_block_column_end),
    #     color = RED,
    #     thickness = 1,
    #     lineType = cv2.LINE_AA,
    # )
    # show(
    #     (f"Local ridge with average ridge frequency = {fingerprint.ridge_frequency}", fingerprint_with_highlighted_ridge_block),
    #     ("Enhanced fingeprint", fingerprint.enhanced_fingerprint),
    #     ("Binarized fingeprint", fingerprint.binarized_fingerprint),
    #     ("Thinned fingeprint", fingerprint.thinned_fingerprint),
    #     ("Directional Map", fingerprint_with_directional_lines),
    # )

    # fingerprint_with_gabor_filters: list[tuple[str, NDArray[u8]]] = []
    # gabor_filters: list[tuple[str, NDArray[f64]]] = []
    # for gabor_kernel, gabor_kernel_angle, fingerprint_with_gabor_filter in zip(
    #     fingerprint.gabor_filters,
    #     fingerprint.gabor_filters_angles,
    #     fingerprint.fingerprint_with_gabor_filters
    # ):
    #     angle_in_degrees = round(gabor_kernel_angle * 180 / numpy.pi, ndigits = 2)
    #     label = f"{angle_in_degrees}"
    #     gabor_filters.append((label, gabor_kernel))
    #     fingerprint_with_gabor_filters.append((label, fingerprint_with_gabor_filter))
    # show(
    #     *gabor_filters,
    #     *fingerprint_with_gabor_filters,
    #     max_images_per_row = gabor_filters_count
    # )

    # singularities = Singularities(
    #     fingerprint.directional_map,
    #     directional_map_block_length,
    #     fingerprint.segmentation_mask_distance_map,
    #     singularities_min_distance_from_border,
    # )
    # minutiae = Minutiae(
    #     fingerprint.thinned_fingerprint,
    #     fingerprint.segmentation_mask_distance_map,
    #     minutiae_min_distance_from_border,
    # )
    # thinned_fingerprint_with_all_minutiae = draw_minutiae(
    #     fingerprint.thinned_fingerprint,
    #     minutiae.all,
    #     RED,
    #     BLUE,
    # )
    # thinned_fingerprint_with_filtered_minutiae = draw_minutiae(
    #     fingerprint.thinned_fingerprint,
    #     minutiae.filtered,
    #     RED,
    #     BLUE,
    # )
    thinned_fingerprint_with_valid_minutiae = draw_minutiae_with_angle(
        fingerprint.thinned_fingerprint,
        fingerprint.minutiae,
        RED,
        BLUE,
    )
    # thinned_fingerprint_with_all_singularities = draw_singularities(
    #     fingerprint.thinned_fingerprint,
    #     singularities.all,
    #     RED,
    #     BLUE,
    #     GREEN,
    # )
    # thinned_fingerprint_with_filtered_singularities = draw_singularities(
    #     fingerprint.thinned_fingerprint,
    #     singularities.filtered,
    #     RED,
    #     BLUE,
    #     GREEN,
    # )
    # thinned_fingerprint_with_valid_singularities = draw_singularities(
    #     fingerprint.thinned_fingerprint,
    #     fingerprint.singularities,
    #     RED,
    #     BLUE,
    #     GREEN,
    # )
    # fingerprint_with_directional_lines = draw_directional_map_lines(
    #     fingerprint.normalized_fingerprint,
    #     fingerprint.directional_map,
    #     fingerprint.segmentation_mask,
    #     directional_map_block_length,
    #     RED,
    # )
    show(
        # ("All minutiae", thinned_fingerprint_with_all_minutiae),
        # ("Filtered minutiae", thinned_fingerprint_with_filtered_minutiae),
        ("Valid minutiae with angles", thinned_fingerprint_with_valid_minutiae),
        # ("All singularities", thinned_fingerprint_with_all_singularities),
        # # ("Filtered singularities", thinned_fingerprint_with_filtered_singularities),
        # ("Valid singularities with angles", thinned_fingerprint_with_valid_singularities),
        # ("Directional Map", fingerprint_with_directional_lines),

        max_images_per_row = 3,
    )


interactive(children=(IntSlider(value=1, description='db tag', layout=Layout(width='auto'), max=4, min=1, styl…

In [5]:
from typing import cast
from features_extraction_utils import YELLOW, ColorBGR, convert_to_bgr_if_grayscale, draw_matching_hough
from fingerprint import Alignment, Minutia, MinutiaWithAngle, Range, TerminationWithAngle, normalize_angle_radians, normalize_pixels
from utils import FINGERPRINTS_DATABASE_FILE_EXTENSION

@ipywidgets.interact(
    identity_db_tag = int_slider(range = db_tag_range, description = "identity db tag"),
    identity_finger_tag = int_slider(range = finger_tag_range, description = "identity finger tag"),
    identity_acquisition_tag = int_slider(range = acquisition_tag_range, description = "identity acquisition tag"),
    template_db_tag = int_slider(range = db_tag_range, description = "template db tag"),
    template_finger_tag = int_slider(range = finger_tag_range, description = "template finger tag"),
    template_acquisition_tag = int_slider(range = acquisition_tag_range, description = "template acquisition tag"),
    matching_score_threshold = float_slider(
        range = FM_CONFIG.matching_score_genuine_threshold,
        description = "matching score",
    ),
    pixels_distance_threshold = int_slider(
        range = FM_CONFIG.hough_matching.pixels_distance_threshold,
        description = "pixels distance threshold",
    ),
    angle_distance_threshold = int_slider(
        range = FM_CONFIG.hough_matching.angle_distance_threshold,
        description = "angle distance threshold",
    ),
    alignment_angle_freedom = int_slider(
        range = FM_CONFIG.hough_matching.alignment_angle_freedom,
        description = "alignment angle freedom",
    ),
    alignment_scale_freedom = int_slider(
        range = FM_CONFIG.hough_matching.alignment_scale_freedom,
        description = "alignment scale freedom",
    ),
) # type: ignore
def matching_score_hough(
    identity_db_tag: int,
    identity_finger_tag: int,
    identity_acquisition_tag: int,
    template_db_tag: int,
    template_finger_tag: int,
    template_acquisition_tag: int,
    matching_score_threshold: float,
    pixels_distance_threshold: int,
    angle_distance_threshold: int,
    alignment_angle_freedom: int,
    alignment_scale_freedom: int,
) -> None:
    identity_database_file_path = f"{DATABASE_DIR_PATH}\\FVC2006\\db{identity_db_tag}_b\\1{identity_finger_tag:02}{FINGERPRINTS_DATABASE_FILE_EXTENSION}"
    template_database_file_path = f"{DATABASE_DIR_PATH}\\FVC2006\\db{template_db_tag}_b\\1{template_finger_tag:02}{FINGERPRINTS_DATABASE_FILE_EXTENSION}"

    identity_database: NDArray[Any] = numpy.load(identity_database_file_path, allow_pickle = True)
    template_database: NDArray[Any] = numpy.load(template_database_file_path, allow_pickle = True)

    identity_fingerprint: Fingerprint = identity_database[identity_acquisition_tag - 1]
    template_fingerprint: Fingerprint = template_database[template_acquisition_tag - 1]

    matching_score, aligned_minutiae, matched_minutiae, alignment = Fingerprint.matching_score_hough(
        identity_fingerprint.minutiae,
        template_fingerprint.minutiae,
        pixels_distance_threshold,
        angle_distance_threshold,
        alignment_angle_freedom,
        alignment_scale_freedom,
    )

    matching_hough = draw_matching_hough(
        identity_fingerprint,
        template_fingerprint,
        matching_score,
        aligned_minutiae,
        matched_minutiae,
        alignment,
        matching_score_threshold,
        0.6,
    )
    show((f"matching hough {matching_score}", matching_hough))


interactive(children=(IntSlider(value=1, description='identity db tag', layout=Layout(width='auto'), max=4, mi…

In [11]:
fingerprint_with_minutiae = draw_minutiae_with_angle(
    fingerprint.thinned_fingerprint,
    fingerprint.minutiae,
    RED,
    BLUE,
)

@ipywidgets.interact(
    minutia_index = int_slider(
        range = Range[int].new(
            min = 0,
            max = len(fingerprint.minutiae) - 1,
            step = 1,
            value = 0,
        ),
        description = "minutia index",
    ),
) # type: ignore
def draw_cilinders(minutia_index: int) -> None:
    fingerprint_with_minutiae_and_cylinders = draw_mcc_cylinders(
        fingerprint_with_minutiae.copy(),
        fingerprint,
        minutia_index
    )
    show((f"cylinder {minutia_index}", fingerprint_with_minutiae_and_cylinders))


interactive(children=(IntSlider(value=0, description='minutia index', layout=Layout(width='auto'), max=16, sty…

In [12]:
other_fingerprint_database_tag = "FVC2006/db1_b/101"
other_fingerprint_database_path = os.path.join(DATABASE_DIR_PATH, other_fingerprint_database_tag) + ".npz"

fingerprints: list[Fingerprint]
with numpy.load(other_fingerprint_database_path, allow_pickle = True) as expected_fingerprints_database:
    expected_fingerprints_database: NDArray[Any]
    fingerprints = expected_fingerprints_database["fingerprints"].tolist() # type: ignore

def matching_score_local_structures(
    acquisition_tag: int,
) -> tuple[float, list[tuple[str, NDArray[u8]]]]:
    other_fingerprint_tag = f"FVC2006/db1_b/101_{acquisition_tag}"
    other_fingerprint_file_path = os.path.join(DATASET_DIR_PATH, other_fingerprint_tag) + FINGERPRINTS_IMAGE_FILE_EXTENSION
    other_fingerprint_file_path = os.path.normpath(other_fingerprint_file_path)
    other_raw_fingerprint: NDArray[u8] = cv2.imread(other_fingerprint_file_path, flags = cv2.IMREAD_GRAYSCALE) # type: ignore

    other = fingerprints[acquisition_tag - 1]

    distances: NDArray[f64] = numpy.linalg.norm(
        fingerprint.acquisition.features.local_structures[:, numpy.newaxis,:] - other.acquisition.features.local_structures,
        axis = -1
    )
    distances /= numpy.linalg.norm(fingerprint.acquisition.features.local_structures, axis = 1)[:, numpy.newaxis] + numpy.linalg.norm(other.acquisition.features.local_structures, axis = 1)
    minutiae_matching_pairs: tuple[NDArray[i64], NDArray[i64]] = numpy.unravel_index(
        numpy.argpartition(
            distances,
            FM_CONFIG.local_structures_matching_minutiae_pair_count, None
        )[: FM_CONFIG.local_structures_matching_minutiae_pair_count],
        distances.shape
    ) # type: ignore
    matching_score = float(1 - numpy.mean(distances[minutiae_matching_pairs[0], minutiae_matching_pairs[1]]))

    fingerprint_with_minutiae = draw_minutiae_with_angle(fingerprint.raw_fingerprint, fingerprint.acquisition.features.minutiae, fingerprint.acquisition.features.minutiae_kinds)
    other_with_minutiae = draw_minutiae_with_angle(other_raw_fingerprint, other.acquisition.features.minutiae, other.acquisition.features.minutiae_kinds)

    fingerprint_columns, fingerprint_rows = fingerprint.raw_fingerprint
    if matching_score >= FM_CONFIG.matching_score_genuine_threshold.value:
        fingerprint_with_minutiae: NDArray[u8] = cv2.rectangle(
            fingerprint_with_minutiae,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = GREEN,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore
    else:
        fingerprint_with_minutiae: NDArray[u8] = cv2.rectangle(
            fingerprint_with_minutiae,
            pt1 = (0, 0),
            pt2 = (fingerprint_columns - 1, fingerprint_rows - 1),
            color = RED,
            thickness = 1,
            lineType = cv2.LINE_AA,
        ) # type: ignore

    match_pairs_images: list[tuple[str, NDArray[u8]]] = []
    for matching_pair_index in range(len(minutiae_matching_pairs[0])):
        match_pairs = draw_match_pairs(
            fingerprint_with_minutiae.copy(),
            fingerprint.acquisition.features,
            other_with_minutiae.copy(),
            other.acquisition.features,
            minutiae_matching_pairs,
            matching_pair_index,
        )

        match_pair_name = f"match pair {matching_pair_index}"
        match_pairs_images.append((match_pair_name, match_pairs))

    return matching_score, match_pairs_images

matching_score, matching_pairs = matching_score_local_structures(
    acquisition_tag = 4,
)

def show_matching_pairs(matching_pair_index: int) -> None:
    show(matching_pairs[matching_pair_index])

show_matching_pairs_widget = ipywidgets.interactive(
    show_matching_pairs,
    matching_pair_index = ipywidgets.IntSlider(
        min = 0,
        max = len(matching_pairs) - 1,
        description = "matching pair index",
        layout = ipywidgets.Layout(width = "auto"),
        style = {"description_width": "initial"}
    ),
)

_ = display(matching_score, show_matching_pairs_widget)

FileNotFoundError: [Errno 2] No such file or directory: 'database\\FVC2006/db1_b/101.npz'